### Import packages

In [4]:
pip install -e ..

Obtaining file:///datadrive/code/ecmwf-forecast
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for stactools-ecmwf-forecast (pyproject.toml) ... done
  Created wheel for stactools-ecmwf-forecast: filename=stactools_ecmwf_forecast-0.1.0-0.editable-py3-none-any.whl size=2701 sha256=b1056658ce5bcebd9832f20a2e81f273d8a3d03f01fb5f2c89db13a073a540f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-znyqk6wg/wheels/3c/b5/ed/198e6195eec3f29011301fb54db9bc02a9d68b2f3012f669fd
Successfully built stactools-ecmwf-forecast
  Attempting uninstall: stactools-ecmwf-forecast
    Found existing installation: stactools-ecmwf-forecast 0.1.0
    Uninstalling stactools-ecmwf-forecast-0.1.0:
      Successfully uninstalled stactools-ecmwf-forecast-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import fsspec
import planetary_computer
import pystac_client
import urllib.request
import xarray as xr
from stactools.ecmwf_forecast import stac

/anaconda/envs/py38_default/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


### Create STAC item

In [8]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
search = catalog.search(
    collections=["ecmwf-forecast"],
    query={
        "ecmwf:stream": {"eq": "wave"},
        "ecmwf:type": {"eq": "fc"},
        "ecmwf:step": {"eq": "0h"},
    },
)
item = next(search.get_items())
link = item.assets['data'].href
account_name = link.split('https://')[1].split('.blob.core.windows.net/')[0]
filepath, sas_token = link.split('?')
conn_string = 'DefaultEndpointsProtocol=https;AccountName={};SharedAccessSignature={}'.format(account_name, sas_token)

('ai4edataeuwest',
 'https://ai4edataeuwest.blob.core.windows.net/ecmwf/20230929/00z/0p4-beta/wave/20230929000000-0h-wave-fc.grib2')

In [9]:
%%time
item = stac.create_item([filepath], connection_string = conn_string)

CPU times: user 602 ms, sys: 60.3 ms, total: 663 ms
Wall time: 2.21 s


/anaconda/envs/py38_default/lib/python3.8/site-packages/kerchunk/combine.py:269: UserWarning: Concatenated coordinate 'valid_time' contains less than expectednumber of values across the datasets: [1695945600]
  warnings.warn(


### Load dataset via kerchunk indices

In [10]:
%%time
m = fsspec.get_mapper('reference://', fo=item.properties['kerchunk_indices'], remote_options={"connection_string": conn_string})
ds = xr.open_dataset(m, engine='zarr')
ds['swh'].mean()

CPU times: user 62.2 ms, sys: 11.8 ms, total: 73.9 ms
Wall time: 184 ms


<timed exec>:2: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.


<xarray.DataArray 'swh' ()>
array(2.55190232)

### Load dataset via original .GRIB2 file

In [13]:
%%time
filename, _ = urllib.request.urlretrieve(filepath)
ds = xr.open_dataset(filename, engine="cfgrib")
ds['swh'].mean()

CPU times: user 957 ms, sys: 43.7 ms, total: 1 s
Wall time: 2.55 s


<xarray.DataArray 'swh' ()>
array(2.551902, dtype=float32)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    meanSea     float64 ...
    valid_time  datetime64[ns] ...